In [1]:
#Julian Felipe Gutiérrez
#Juan David Marin

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Read data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# It defines the regression model
def inference(x):
    weight_init = tf.random_normal_initializer()
    bias_init = tf.constant_initializer(value=0)
    W = tf.get_variable("W", [784, 10], initializer=weight_init)
    b = tf.get_variable("b", [10], initializer=bias_init)
    output = tf.nn.softmax(tf.matmul(x, W) + b)
    return output

In [ ]:
# It defines the function of loss associated to the model
def loss(output, y):
    dot_product = y * tf.log(output)    
    xentropy = -tf.reduce_sum(dot_product, reduction_indices=1)
    loss = tf.reduce_mean(xentropy)
    return loss

In [ ]:
# It defines the accuracy of the predictions to make from the model
def evaluate(output, y):
    correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

In [ ]:
# It trains the model
def training(cost, global_step):
    tf.summary.scalar("cost", cost)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op

In [ ]:
# Setting of parameters

learning_rate = 0.01
training_epochs = 30
batch_size = 100
display_step = 1

with tf.Graph().as_default():
    # mnist data image of shape 28*28=784
    x = tf.placeholder(tf.float32, [None, 784])
    # mnist data image of shape 16*16=256
    x = tf.placeholder(tf.float32, [784, 256])
    # mnist data image of shape 16*16=256
    x = tf.placeholder(tf.float32, [256, 256])
    # 0-9 digits recognition => 10 classes
    y = tf.placeholder(tf.float32, [None, 10])    
    output = inference(x)    
    cost = loss(output, y) 
    
    global_step = tf.Variable(0, name='global_step', trainable=False)    
    train_op = training(cost, global_step)    
    eval_op = evaluate(output, y)
    
    summary_op = tf.summary.merge_all()    
    saver = tf.train.Saver()
    
    sess = tf.Session()    
    #summary_writer = tf.summary.FileWriter("logistic_logs/", graph=sess.graph)
    
    init_op = tf.global_variables_initializer()    
    sess.run(init_op)
    
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            mbatch_x, mbatch_y = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            feed_dict = {x : mbatch_x, y : mbatch_y}
            sess.run(train_op, feed_dict=feed_dict)
            # Compute average loss
            minibatch_cost = sess.run(cost, feed_dict=feed_dict)
            avg_cost += minibatch_cost/total_batch
            
        # Display logs per epoch step
        if epoch % display_step == 0:
            val_feed_dict = {
                x : mnist.validation.images,
                y : mnist.validation.labels
            }
            accuracy = sess.run(eval_op, feed_dict=val_feed_dict)
            print ("Validation Error:", (1 - accuracy))
            summary_str = sess.run(summary_op, feed_dict=feed_dict)
            #summary_writer.add_summary(summary_str, sess.run(global_step))
            
            saver.save(sess, "logistic_logs2/model-checkpoint", global_step=global_step)
            
    print("Optimization Finished!")
    
    test_feed_dict = {
        x : mnist.test.images,
        y : mnist.test.labels
    }
    accuracy = sess.run(eval_op, feed_dict=test_feed_dict)
    writer = tf.summary.FileWriter("logistic_logs2/", graph=sess.as_default)
    writer.add_graph(tf.get_default_graph())
    print("Test Accuracy:", accuracy)